#Introduce

This notebook is created for the case using to load the Dataset (DIV2k) with difference key, and preprocessing image:


1.   Load the dataset with key.
2.   Preprocessing image.
3.   Seperate the dataset.



In [2]:
print('import libary...')
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
  

import libary...


In [ ]:
def load_dataset(key='div2k',include_validate=True):
  print('using tensorflow_datasets to load the dataset','\nloading dataset...')
  if include_validate:
    (train_ds, val_ds), metadata = tfds.load(
    key,
    split=['train','validation'],
    with_info=True,
    as_supervised=True,
   )
  else:
    (train_ds), metadata = tfds.load(
    key,
    split=['train'],
    with_info=True,
    as_supervised=True,
    )
  print(metadata)
  return (train_ds,val_ds),metadata

In [ ]:
def Load_dataset_from_directory(path,scale=4):
  def process_path(file_path):
  # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(img, channels=3)
    return img
  list_ds = tf.data.Dataset.list_files(path+'/*', shuffle=False)
  data_length = len(list_ds)
  hr_data = list_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
  def build_pair_image(data,scale=4):
    lr=[]
    for img in data.take(len(data)):
      lr.append(tf.cast(tf.image.resize(img, [int(img.shape[0]/scale),int(img.shape[1]/scale)], method=tf.image.ResizeMethod.BICUBIC),dtype=tf.int32))
    return lr
  lr_data = tf.data.Dataset.from_generator(lambda: 
                              build_pair_image(hr_data),
                              output_types=( tf.int32),
                              output_shapes=(tf.TensorShape([ None, None, 
                                         3])))
  data = tf.data.Dataset.zip((lr_data,hr_data))
  data=data.apply(tf.data.experimental.assert_cardinality(data_length))
  return data

In [ ]:
def normalize_img(image):
  print('Normalizes images: `uint8` -> `float32`')
  return tf.cast(image, tf.float32) /255

In [ ]:
def configure_performan(ds,BUFFER_SIZE=1000,BATCH_SIZE=16):
  ds=ds.cache()
  ds=ds.shuffle(buffer_size=BUFFER_SIZE)
  ds=ds.batch(BATCH_SIZE)
  ds=ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  return ds

In [75]:
def show(image, label):
  plt.figure(figsize=(len(image.numpy())/100,len(image.numpy()[0])/100))
  plt.imshow(image)
  plt.title(np.array(label))
  plt.axis('on')

In [ ]:
def load_image_from_path(path):
  image = tf.keras.preprocessing.image.load_img(path)
  image = tf.keras.preprocessing.image.img_to_array(image)
  image = tf.constant(image)
  return tf.cast(image,tf.int32)

In [ ]:
def save_image_path(path,image):
  tf.keras.utils.save_img(
    path, image
  )

In [ ]:
def visualize_augmented(original, augmented,type='lr'):
  # plt.figure(figsize=(len(original.numpy())/80,len(original.numpy()[0])/80))
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image'+type)
  plt.imshow(original)
  plt.subplot(1,2,2)
  plt.title('Augmented image'+type)
  plt.imshow(augmented)

In [ ]:
def augment(lr,hr,with_adjust=True):
  #get the same seed to get the same operator to both lr and hr for image when we use randomly
  generator= tf.random.get_global_generator()
  seed= generator.make_seeds(2)[0]
  #mean that change the pixel value by adujst it's factor
  if with_adjust==True:
    #low resolution augmentation
    lr=tf.image.stateless_random_contrast(lr,0.8,1.2,seed)
    lr=tf.image.stateless_random_brightness(lr,0.10,seed)
    lr=tf.image.stateless_random_saturation(lr,0.8,1.3,seed)
    #high resolution augmentation
    hr=tf.image.stateless_random_contrast(hr,0.8,1.2,seed)
    hr=tf.image.stateless_random_brightness(hr,0.10,seed)
    hr=tf.image.stateless_random_saturation(hr,0.8,1.3,seed)
  #random rotate and flip
  roate_factor=tf.squeeze(generator.uniform([1],minval=1,maxval=5,dtype=tf.dtypes.int32), axis=0)
  lr=tf.image.rot90(lr,roate_factor)
  lr=tf.image.stateless_random_flip_left_right(lr,seed)
  hr=tf.image.rot90(hr,roate_factor)
  hr=tf.image.stateless_random_flip_left_right(hr,seed)
  return lr,hr

In [ ]:
def random_crop(lr_img, hr_img, hr_crop_size, scale):
    lr_crop_size = hr_crop_size // scale
    lr_shape = tf.shape(lr_img)[:2]
    lr_top = tf.random.uniform(shape=(), maxval=lr_shape[0] - lr_crop_size + 1, dtype=tf.int32)
    lr_left = tf.random.uniform(shape=(), maxval=lr_shape[1] - lr_crop_size + 1, dtype=tf.int32)
    hr_top = lr_top * scale
    hr_left = lr_left * scale
    lr_crop = lr_img[lr_top:lr_top + lr_crop_size, lr_left:lr_left + lr_crop_size]
    hr_crop = hr_img[hr_top:hr_top + hr_crop_size, hr_left:hr_left + hr_crop_size]
    return lr_crop, hr_crop

In [ ]:
# Normalizes RGB images to [-1, 1].
def normalize_m11(x):
    return x / 127.5 - 1
# Normalizes RGB images to [0, 1].
def normalize_01(x):
    return x / 255.0
# Denomalizes RGB images from [-1, 1].
def denormalize_m11(x):
    return (x + 1) * 127.5

In [ ]:
def gaussian_noise_layer(input_layer, std):
    noise = tf.random.normal(shape=tf.shape(input_layer), mean=0.0, stddev=std, dtype=tf.float32) 
    return input_layer + noise

In [ ]:
def gaussian_kernel(size: int,
                    mean: float,
                    std: float,
                   ):
    """Makes 2D gaussian Kernel for convolution."""

    d = tf.distributions.Normal(mean, std)

    vals = d.prob(tf.range(start = -size, limit = size + 1, dtype = tf.float32))

    gauss_kernel = tf.einsum('i,j->ij',
                                  vals,
                                  vals)

    return gauss_kernel / tf.reduce_sum(gauss_kernel)

In [ ]:
# gauss_kernel = gaussian_kernel( ... )

# # Expand dimensions of `gauss_kernel` for `tf.nn.conv2d` signature.
# gauss_kernel = gauss_kernel[:, :, tf.newaxis, tf.newaxis]

# # Convolve.
# tf.nn.conv2d(image, gauss_kernel, strides=[1, 1, 1, 1], padding="SAME")

#Visualize Sample

Download evaluate dataset included:


*   Set5
*   Set14
* BSD100



In [ ]:
# !wget --no-check-certificate \
#     https://data.deepai.org/set5.zip \
#     -O /content/drive/MyDrive/Dataset/Evaludate/ds.zip
  

In [ ]:
# import os
# import zipfile
# local_zip = '/content/drive/MyDrive/Dataset/Evaludate/ds.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('/content/drive/MyDrive/Dataset/Evaludate/')
# zip_ref.close()
# local_zip = '/content/drive/MyDrive/Dataset/Evaludate/set5/SR_testing_datasets.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('/content/drive/MyDrive/Dataset/Evaluate_dataset')
# zip_ref.close()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# ! pwd
# %cd drive/MyDrive
# %cd Colab-Super_resolution/
# %cd /content/drive/MyDrive/Dataset